In [22]:
import pandas as pd
import numpy as np

## Import and tidy up Weather data

In [23]:
wr = pd.read_csv("../raw-data/weather_history.csv", low_memory=False) 

In [24]:
wr = wr.dropna(how='all', axis=1)
wr = wr.drop('WW', axis=1)
wr = wr.rename(columns={ wr.columns[0]: "Time" })

In [25]:
wr = wr.rename(columns={'T':'Temp','U':'Rel_Humidity','DD':'Wind_Dir',
                        'Tn':'Min_Temp','Tx':'Max_Temp','Td':'Dew_Temp',
                        'RRR':'Rain','tR':'Rain_Time','Tg':'Night_Soil_Temp'})
wr = wr.drop(['Dew_Temp','Night_Soil_Temp','Min_Temp','Max_Temp','Wind_Dir'], axis=1)

In [26]:
wr['Rain'].replace('No precipitation',0, inplace=True)
wr['Rain'] = pd.to_numeric(wr['Rain'])
wr['Hourly_Rain'] = wr['Rain'] / wr['Rain_Time']
wr = wr.drop(['Rain','Rain_Time'], axis = 1) # unclear why Rain_Time is sometimes 6 and sometimes 12
wr['Time'] = pd.to_datetime(wr['Time'], dayfirst=True)

In [27]:
wr['Hour'] = wr['Time'].dt.hour
wr['Date'] = wr['Time'].dt.date

## Aggregate weather by day

In [28]:
wrd = wr.groupby('Date').mean().reset_index()
wrd = wrd.drop('Hour', axis=1)
wrd['Date'] = pd.to_datetime(wrd['Date'])
wrd['Date'] = wrd['Date'].dt.date

## Import Daily Cycle Hire data

In [29]:
dh = pd.read_csv("../raw-data/daily-hires.csv")
dh['Date'] = pd.to_datetime(dh['Date'], dayfirst=True).dt.date
dh['Hires']=dh['Hires'].astype(int)

### Import strike data

In [30]:
strikes = pd.read_csv('../raw-data/tube-strikes.csv')
strikes['Strike_Evening'] = pd.to_datetime(strikes['Strike_Evening'], dayfirst=True)
strikes['Strike_Daytime'] = pd.to_datetime(strikes['Strike_Daytime'], dayfirst=True)

### Import bank holidays

In [31]:
bank = pd.read_csv('../raw-data/bank-holidays.csv')
bank['Date'] = pd.to_datetime(bank['Date'], dayfirst=True)

### Import major London cycle accidents

In [32]:
cyc = pd.read_csv('../raw-data/cycle-deaths-injuries.csv')

In [33]:
death = cyc[cyc['Type']=='death']
death = list(death['Date'])
inj = cyc[cyc['Type']=='injury']
inj = list(inj['Date'])


## Import network size data

In [34]:
nw = pd.read_csv("../raw-data/hire-bike-volumes.csv")

nw = nw.rename(columns={'Stations':'Dock_Points'})
nw['Date'] = pd.to_datetime(nw['Date'], dayfirst=True).dt.date

#impute missing values using last known or estimated value
nw = nw.fillna(method='ffill')
nw =nw.replace({'Registration': {'Yes': True, 'No': False},
               'Weekly_Fee':{'y': True, 'n': False}})

### Create dataframe of previous and future dates

In [35]:
dates = [d for d in pd.date_range('20100730','20201231')] # until end of 2020
dates = pd.DataFrame(dates, columns=['Date'])
dates['Date'] = pd.to_datetime(dates['Date']).dt.date

### Join Daily Cycle Hire, Aggregated Daily Weather, and Network Size data

In [36]:
daily = dates.merge(dh, how='left', on='Date')
daily = daily.merge(wrd, how='left', on='Date')
daily = daily.merge(nw, how='left', on='Date')
daily['Date'] = pd.to_datetime(daily['Date'])
daily = daily.sort_values(by="Date")

In [37]:
fill_cols = ['Bicycles','Dock_Points','Registration','Sponsor','Fee','Weekly_Fee','Mobike','Ofo','Obike','Urbo']
daily[fill_cols] = daily[fill_cols].fillna(method='ffill')
daily['Comp_bikes'] = daily[['Mobike','Ofo','Obike','Urbo']].sum(axis=1)
daily['hpb'] = daily['Hires'] / daily['Bicycles'] #hires per bike

### Add strikes, bank holidays and accidents

In [38]:
daily['Strike_Evening'] = daily['Date'].isin(strikes['Strike_Evening'])
daily['Strike_Daytime'] = daily['Date'].isin(strikes['Strike_Daytime'])
daily['Strike'] = (daily['Strike_Evening']) | (daily['Strike_Daytime'])
daily['Bank_Hol'] = daily['Date'].isin(bank['Date'])

In [39]:
daily['Death'] = daily['Date'].isin(death)
daily['Inj'] = daily['Date'].isin(inj)
daily['KSI'] = (daily['Death']) | (daily['Inj'])
daily['KSI_day_before'] = daily['KSI'].shift(1)

### Derive extra columns

In [40]:
daily['Month'] = daily['Date'].dt.month
daily['Year'] = daily['Date'].dt.year
daily['Day_of_Week'] = daily['Date'].dt.weekday
daily['Weekend'] = daily['Day_of_Week'] > 4
daily['Day_of_Year'] = daily['Date'].dt.dayofyear

daily['Wkend_or_Hol'] = (daily['Bank_Hol']) | (daily['Weekend'])
daily['Date'] = daily['Date'].dt.date
daily = daily.sort_values(by="Date")
days=["Mon","Tues","Wed","Thu","Fri","Sat","Sun"]


def day(number):
    return days[number]

daily['Day'] = daily['Day_of_Week'].apply(day)

### Import and add daylight hours

In [41]:
light = pd.read_csv('../raw-data/civil-twilight-2017.csv')

light['Dark'] = pd.to_datetime(light['Dark'],format= '%H:%M')
light['Light'] = pd.to_datetime(light['Light'],format= '%H:%M')

light['Hours_Light'] = (light['Dark'] - light['Light'])
light['Dark'] = light['Dark'].dt.time
light['Light'] = light['Light'].dt.time

light['Hours_Light'] = (light['Hours_Light'].dt.seconds) / 3600

light['Day_of_Year'] = pd.to_datetime(light['Date']).dt.dayofyear
light = light.drop(['Date'], axis=1)

daily = daily.merge(light, how='left', on='Day_of_Year')

In [42]:
daily.to_csv('../data/daily.csv', index=False)